In [1]:
import pandas as pd 
from nltk.corpus import stopwords
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch
import numpy as np
from numpy import argmax
from torch.nn.functional import softmax

In [2]:
model_name = "ProsusAI/finbert"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [44]:
df = pd.read_csv('FinalStockNews.csv')
for col in df.columns:
    col = col.strip
df.drop('Unnamed: 0', axis=1, inplace=True)
df.dropna(inplace=True)
df.reset_index(inplace=True, drop=True)
df

,Unnamed: 0.1,ticker,title,date,link,articleInfo,paragraphList
0,0,AMZN,"Tech Layoffs, Remote Work Push Office Vacancie...",2024-04-16T00:00:00,/news/etf/tech-layoffs-remote-work-push-office...,The commercial real estate industry continues ...,['The commercial real estate industry continue...
1,1,AMZN,The 3 Best Stocks to Buy With Your Tax Refund ...,2024-04-16T00:00:00,/news/stocks/the-3-best-stocks-to-buy-with-you...,The IRS typically gives tax refunds within 21 ...,"['InvestorPlace - Stock Market News, Stock Adv..."
2,2,AMZN,3 AI Stocks to Buy Now: Q2 Edition,2024-04-16T00:00:00,/news/stocks/3-ai-stocks-to-buy-now-q2-edition...,Just like electricity and the internet unleash...,"['InvestorPlace - Stock Market News, Stock Adv..."
3,3,AMZN,"Amazon, Meta, Uber Remain 'Top Overall Picks' ...",2024-04-16T00:00:00,/news/stocks/amazon-meta-uber-remain-top-overa...,As the first-quarter (Q1) 2024 earnings season...,['As the first-quarter (Q1) 2024 earnings seas...
4,4,AMZN,Amazon (AMZN) Receives a Buy from J.P. Morgan,2024-04-16T00:00:00,/news/stocks/amazon-amzn-receives-a-buy-from-j...,J.P. Morgan analyst Doug Anmuth maintained a B...,['J.P. Morgan analyst Doug Anmuth maintained a...
...,...,...,...,...,...,...,...
1559,1559,AAPL,Riding the Bull: 3 ETFs to Capitalize on the O...,2024-03-13T00:00:00,/news/stocks/riding-the-bull-3-etfs-to-capital...,There are many advantages to owing exchange-tr...,"['InvestorPlace - Stock Market News, Stock Adv..."
1560,1560,AAPL,Apple’s Bitter Bite: Is the Tech Giant’s Slowi...,2024-03-13T00:00:00,/news/stocks/apples-bitter-bite-is-the-tech-gi...,The market has shown extreme volatility for mo...,"['InvestorPlace - Stock Market News, Stock Adv..."
1561,1561,AAPL,SCHD ETF Spotlight: The Top 6 Stocks Inside Th...,2024-03-13T00:00:00,/news/stocks/schd-etf-spotlight-the-top-6-stoc...,Investing in the stock market is one of the su...,"['InvestorPlace - Stock Market News, Stock Adv..."
1562,1562,AAPL,Maintaining Market-Perform on Apple: A Balance...,2024-03-13T00:00:00,/news/stocks/maintaining-market-perform-on-app...,Bernstein analyst Toni Sacconaghi maintained a...,['Bernstein analyst Toni Sacconaghi maintained...


In [6]:
def sliding_window(text, tokenizer, max_length=512, overlap=50):

    #encodes text
    tokenized_text = tokenizer.encode_plus(text, return_tensors='pt', add_special_tokens=True)
    input_ids = tokenized_text['input_ids'].squeeze().numpy().tolist()
    
    total_length = len(input_ids)
    #each step is 462 by default
    step_size = max_length - overlap
    chunks = []
    
    #create sliding window and tokenize each chunk
    for start in range(0, total_length, step_size):
        end = start + max_length
        chunk = input_ids[start:end]
        
        chunk = [tokenizer.cls_token_id] + chunk + [tokenizer.sep_token_id]
        chunks.append(chunk)
    
    return chunks


In [7]:
def process_chunks(chunks, tokenizer, model):
    # Set the model to evaluation mode
    model.eval()  
    
    # Lists to store sentiments and scores
    sentiments = []
    scores = []
    
    # Disable gradient calculation for efficiency
    with torch.no_grad():
        # Iterate through each chunk
        for chunk in chunks:
            # Convert the chunk into a PyTorch tensor and add a batch dimension
            inputs = torch.tensor(chunk).unsqueeze(0)  
            
            # Pad the input tensor to match the model's input size
            inputs = torch.nn.functional.pad(inputs, (0, 512 - inputs.shape[1]), value=tokenizer.pad_token_id)
            # Pass the input tensor through the model
            outputs = model(inputs)
            # Retrieve the logits (raw outputs) from the model
            logits = outputs.logits
            # Determine the sentiment by selecting the index of the maximum logit value
            sentiment = torch.argmax(logits, dim=1).numpy()[0]  
            # Calculate the softmax probability score for the predicted sentiment
            score = torch.softmax(logits, dim=1).max().item()  
            # Append the sentiment and score to their respective lists
            sentiments.append(sentiment)
            scores.append(score)

    # Return the lists of sentiments and scores
    return sentiments, scores


In [8]:
def aggregate_results(sentiments, scores):
    # Dictionary to store average score per sentiment
    average_score_per_sentiment = {}
    # Iterate through each sentiment and score pair
    for sentiment, score in zip(sentiments, scores):
        # If sentiment not in the dictionary, add it with an empty list
        if sentiment not in average_score_per_sentiment:
            average_score_per_sentiment[sentiment] = []
        # Append the score to the list of scores for this sentiment
        average_score_per_sentiment[sentiment].append(score)
    
    # Calculate the average score for each sentiment
    for sentiment in average_score_per_sentiment:
        average_score_per_sentiment[sentiment] = sum(average_score_per_sentiment[sentiment]) / len(average_score_per_sentiment[sentiment])
    
    # Find the dominant sentiment (the one with the highest average score)
    dominant_sentiment = max(average_score_per_sentiment, key=average_score_per_sentiment.get)
    
    # Return the dominant sentiment and its average score
    return dominant_sentiment, average_score_per_sentiment[dominant_sentiment]

In [9]:
def full_window(text, tokenizer, model):
    # Generate chunks of text using a sliding window approach
    chunks = sliding_window(text, tokenizer)
    # Process the chunks to obtain sentiments and scores
    sentiments, scores = process_chunks(chunks, tokenizer, model)
    # Aggregate the results to find the dominant sentiment and its average score
    return aggregate_results(sentiments, scores)

In [11]:
df['sentiment'] = df['articleInfo'].apply(lambda news: full_window(news, tokenizer, model)[0])
df['sentiment probability']  = df['articleInfo'].apply(lambda news: full_window(news, tokenizer, model)[1])
df_sentiment = df.copy()

Token indices sequence length is longer than the specified maximum sequence length for this model (935 > 512). Running this sequence through the model will result in indexing errors
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.


In [37]:
df_sentiment

,Unnamed: 0.1,ticker,title,date,link,articleInfo,paragraphList,sentiment,sentiment probability
0,0,AMZN,"Tech Layoffs, Remote Work Push Office Vacancie...",2024-04-16,/news/etf/tech-layoffs-remote-work-push-office...,The commercial real estate industry continues ...,['The commercial real estate industry continue...,1,0.941901
1,1,AMZN,The 3 Best Stocks to Buy With Your Tax Refund ...,2024-04-16,/news/stocks/the-3-best-stocks-to-buy-with-you...,The IRS typically gives tax refunds within 21 ...,"['InvestorPlace - Stock Market News, Stock Adv...",0,0.587049
2,2,AMZN,3 AI Stocks to Buy Now: Q2 Edition,2024-04-16,/news/stocks/3-ai-stocks-to-buy-now-q2-edition...,Just like electricity and the internet unleash...,"['InvestorPlace - Stock Market News, Stock Adv...",2,0.693048
3,3,AMZN,"Amazon, Meta, Uber Remain 'Top Overall Picks' ...",2024-04-16,/news/stocks/amazon-meta-uber-remain-top-overa...,As the first-quarter (Q1) 2024 earnings season...,['As the first-quarter (Q1) 2024 earnings seas...,0,0.875274
4,4,AMZN,Amazon (AMZN) Receives a Buy from J.P. Morgan,2024-04-16,/news/stocks/amazon-amzn-receives-a-buy-from-j...,J.P. Morgan analyst Doug Anmuth maintained a B...,['J.P. Morgan analyst Doug Anmuth maintained a...,2,0.800178
...,...,...,...,...,...,...,...,...,...
1559,1559,AAPL,Riding the Bull: 3 ETFs to Capitalize on the O...,2024-03-13,/news/stocks/riding-the-bull-3-etfs-to-capital...,There are many advantages to owing exchange-tr...,"['InvestorPlace - Stock Market News, Stock Adv...",2,0.797061
1560,1560,AAPL,Apple’s Bitter Bite: Is the Tech Giant’s Slowi...,2024-03-13,/news/stocks/apples-bitter-bite-is-the-tech-gi...,The market has shown extreme volatility for mo...,"['InvestorPlace - Stock Market News, Stock Adv...",2,0.652824
1561,1561,AAPL,SCHD ETF Spotlight: The Top 6 Stocks Inside Th...,2024-03-13,/news/stocks/schd-etf-spotlight-the-top-6-stoc...,Investing in the stock market is one of the su...,"['InvestorPlace - Stock Market News, Stock Adv...",2,0.691881
1562,1562,AAPL,Maintaining Market-Perform on Apple: A Balance...,2024-03-13,/news/stocks/maintaining-market-perform-on-app...,Bernstein analyst Toni Sacconaghi maintained a...,['Bernstein analyst Toni Sacconaghi maintained...,2,0.805873


In [25]:
df.to_csv('dfSentiment.csv')